<a href="https://colab.research.google.com/github/nicoRomeroCuruchet/DynamicProgramming/blob/main/testing_bary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle
import numpy as np
import gymnasium as gym
from PolicyIteration import PolicyIteration 

def get_optimal_action(state:np.array, optimal_policy:PolicyIteration):
    """
    Aproximate the optimal action for a given state using the provided optimal policy
    with barycentric interpolation.

    Parameters:
    state (np.array): The state for which to determine the optimal action.
    optimal_policy (PolicyIteration): The optimal policy used to determine the action.

    Returns:
    action: The optimal action for the given state.
    """
    _, neighbors  = optimal_policy.kd_tree.query([state], k=optimal_policy.num_simplex_points)
    simplex       = optimal_policy.points[neighbors[0]]
    lambdas       = optimal_policy.barycentric_coordinates(state, simplex)

    actions = optimal_policy.action_space
    probabilities = np.zeros(len(actions))

    for i, l in enumerate(lambdas):
        for j, action in enumerate(actions):
            if optimal_policy.policy[tuple(simplex[i])][action] > 0:
                probabilities[j] += l

    argmax = lambda x: max(enumerate(x), key=lambda x: x[1])[0]
    action = actions[argmax(probabilities)]

    return action


def test_enviroment(task: gym.Env, 
                    pi: PolicyIteration, 
                    num_episodes: int = 10000, 
                    episode_lengh: int = 1000):
    """
    Test the environment using the given policy iteration algorithm.

    Parameters:
    - task (gym.Env): The environment to test.
    - pi (PolicyIteration): The policy iteration algorithm.
    - num_episodes (int): The number of episodes to run. Default is 10000.
    - episode_lengh (int): The maximum length of each episode. Default is 1000.
    """

    for episode in range(0, num_episodes):
        total_reward = 0
        observation, _ = task.reset()
        for timestep in range(1, episode_lengh):
            action = get_optimal_action(observation, pi)
            observation, reward, terminated, _, _ = task.step(action)
            total_reward += reward
            if terminated:
                print(f"Episode {episode} finished after {timestep} timesteps")
                print(f"Total reward: {total_reward}")
                break

In [ ]:
# Train cartpole environment:

from classic_control.cartpole import CartPoleEnv

env = CartPoleEnv(sutton_barto_reward=True)
# position thresholds:
x_lim = 2.5
theta_lim = 0.25 
# velocity thresholds:
x_dot_lim = 2.5
theta_dot_lim = 2.5

bins_space = {
    "x_space": np.linspace(-x_lim, x_lim, 20),                         # position space         (0)
    "x_dot_space": np.linspace(-x_dot_lim, x_dot_lim, 20),             # velocity space         (1)
    "theta_space": np.linspace(-theta_lim, theta_lim, 20),             # angle space            (2)
    "theta_dot_space": np.linspace(-theta_dot_lim, theta_dot_lim, 20), # angular velocity space (3)
}

pi = PolicyIteration(
    env=env, 
    bins_space=bins_space,
    action_space=[0, 1],
    gamma=0.99,
    theta=1e-3
)

pi.run()

In [ ]:
# Test cartpole environment:

with open(env.__class__.__name__ + ".pkl", "rb") as f:
    pi = pickle.load(f)

test_enviroment(CartPoleEnv(sutton_barto_reward=True, render_mode="human"), 
                pi)

In [ ]:
# Train mountain car environment:

from classic_control.continuous_mountain_car import Continuous_MountainCarEnv

env=Continuous_MountainCarEnv()

bins_space = {
    "x_space":     np.linspace(env.min_position, env.max_position, 250),      # position space         (0)
    "x_dot_space": np.linspace(-abs(env.max_speed), abs(env.max_speed), 250), # velocity space         (1)
}

pi = PolicyIteration(
    env=env, 
    bins_space=bins_space,
    action_space=[-1, 1],
    gamma=0.99,
    theta=1e-3,
)

pi.run()

In [ ]:
# Test mountain car environment:

with open(env.__class__.__name__ + ".pkl", "rb") as f:
    pi = pickle.load(f)

test_enviroment(Continuous_MountainCarEnv(render_mode="human"), 
                pi)

train acrobot environment:

| Num | Observation                  | Min                 | Max               |
|-----|------------------------------|---------------------|-------------------|
| 0   | Cosine of `theta1`           | -1                  | 1                 |
| 1   | Sine of `theta1`             | -1                  | 1                 |
| 2   | Cosine of `theta2`           | -1                  | 1                 |
| 3   | Sine of `theta2`             | -1                  | 1                 |
| 4   | Angular velocity of `theta1` | ~ -12.567 (-4 * pi) | ~ 12.567 (4 * pi) |
| 5   | Angular velocity of `theta2` | ~ -28.274 (-9 * pi) | ~ 28.274 (9 * pi) |


| Num | Action                                | Unit         |
|-----|---------------------------------------|--------------|
| 0   | apply -1 torque to the actuated joint | torque (N m) |
| 1   | apply 0 torque to the actuated joint  | torque (N m) |
| 2   | apply 1 torque to the actuated joint  | torque (N m) |

In [ ]:
from classic_control.acrobot import AcrobotEnv

env = AcrobotEnv()

bins_space = { "cosine_theta1_space": np.linspace(-1, 1,5), # cos(theta1) space (0)
                "sine_theta1_space": np.linspace(-1, 1, 5),  # sin(theta1) space (1)
                "cosine_theta2_space": np.linspace(-1, 1, 5), # cos(theta2) space (2)
                "sine_theta2_space": np.linspace(-1, 1, 5),   # sin(theta2) space (3)
                "angular_velocity1_space": np.linspace(-env.MAX_VEL_1, env.MAX_VEL_1, 5), # angular velocity 1 space (4)
                "angular_velocity2_space": np.linspace(-env.MAX_VEL_2, env.MAX_VEL_2, 5)  # angular velocity 2 space (5)
            }

pi = PolicyIteration(
    env=env, 
    bins_space=bins_space,
    action_space=[-1, 0, 1],
    gamma=0.99,
    theta=1e-3,
)

pi.run()

| Num | Observation      | Min  | Max |
|-----|------------------|------|-----|
| 0   | x = cos(theta)   | -1.0 | 1.0 |
| 1   | y = sin(theta)   | -1.0 | 1.0 |
| 2   | Angular Velocity | -8.0 | 8.0 |

| Num | Action | Min  | Max |
|-----|--------|------|-----|
| 0   | Torque | -2.0 | 2.0 |


In [22]:
# train pendulum environment:
from classic_control.pendulum import PendulumEnv

env = PendulumEnv()

bins_space = {"theta": np.linspace(0, 2*np.pi, 100), # angle space (0)
              "theta_dot": np.linspace(-1.0, 1.0, 100) # angular velocity space (1)
             } 

pi = PolicyIteration(
    env=env, 
    bins_space=bins_space,
    action_space=list(np.linspace(-2.0, 2.0, 20)),
    gamma=0.99,
    theta=1e-3,
)

pi.run()

2024-05-12 20:34:20.947 | INFO     | PolicyIteration:__init__:96 - Policy Iteration was correctly initialized.
2024-05-12 20:34:20.948 | INFO     | PolicyIteration:__init__:97 - The enviroment name is: PendulumEnv
2024-05-12 20:34:20.948 | INFO     | PolicyIteration:__init__:98 - The action space is: [-2.0, -1.7894736842105263, -1.5789473684210527, -1.368421052631579, -1.1578947368421053, -0.9473684210526316, -0.736842105263158, -0.5263157894736843, -0.3157894736842106, -0.10526315789473695, 0.10526315789473673, 0.3157894736842106, 0.5263157894736841, 0.7368421052631575, 0.9473684210526314, 1.1578947368421053, 1.3684210526315788, 1.5789473684210522, 1.789473684210526, 2.0]
2024-05-12 20:34:20.949 | INFO     | PolicyIteration:__init__:99 - Number of states: 10000
2024-05-12 20:34:20.949 | INFO     | PolicyIteration:run:263 - Generating transition and reward function table...
100%|██████████| 10000/10000 [03:20<00:00, 49.96it/s]
2024-05-12 20:37:41.130 | INFO     | PolicyIteration:run:26

KeyboardInterrupt: 

In [21]:
# test pendulum environment:
with open(env.__class__.__name__ + ".pkl", "rb") as f:
    pi = pickle.load(f)

test_enviroment(PendulumEnv(render_mode="human"),
                pi)

KeyboardInterrupt: 

In [ ]:
#requirements:
#numpy
#tqdm 
#gymnasium
#scipy
#loguru